In [1]:
import numpy as np
from cellpose import models, core, io
from spotiflow.model import Spotiflow
from pathlib import Path
from pathlib import Path
import napari
from utils import list_images, read_image

io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)
spotiflow_model = Spotiflow.from_pretrained("general")



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




c:\Users\adiez_cmic\miniforge3\envs\hcs_cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-09-01 10:24:16,492 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2025-09-01 10:24:16,492 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0
2025-09-01 10:24:16,539 [INFO] ** TORCH CUDA version installed and working. **
2025-09-01 10:24:16,539 [INFO] ** TORCH CUDA version installed and working. **
2025-09-01 10:24:16,539 [INFO] >>>> using GPU (CUDA)
2025-09-01 10:24:17,415 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cpsam
INFO:spotiflow.model.spotiflow:Loading pretrained model: general
2025-09-01 10:24:17,847 [INFO] Loading pretrained model: general


In [5]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("X:\Lisa\siMtb screen I_LØ\Plate 01_Nuc")

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

images

['X:\\Lisa\\siMtb screen I_LØ\\Plate 01_Nuc\\Plate01_Nuc_Wells-A10__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'X:\\Lisa\\siMtb screen I_LØ\\Plate 01_Nuc\\Plate01_Nuc_Wells-A11__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'X:\\Lisa\\siMtb screen I_LØ\\Plate 01_Nuc\\Plate01_Nuc_Wells-A12__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'X:\\Lisa\\siMtb screen I_LØ\\Plate 01_Nuc\\Plate01_Nuc_Wells-A1__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'X:\\Lisa\\siMtb screen I_LØ\\Plate 01_Nuc\\Plate01_Nuc_Wells-A2__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'X:\\Lisa\\siMtb screen I_LØ\\Plate 01_Nuc\\Plate01_Nuc_Wells-A3__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'X:\\Lisa\\siMtb screen I_LØ\\Plate 01_Nuc\\Plate01_Nuc_Wells-A4__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2',
 'X:\\Lisa\\siMtb screen I_LØ\\Plate 01_Nuc\\Plate01_Nuc_Wells-A5__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR.nd2

In [3]:
viewer = napari.Viewer(ndisplay=2)

In [9]:
from tqdm import tqdm

In [ ]:
#TODO: Substract uneven and remove background from BF by obtaining the median of all BF channels 

# Create an empty list to store the brightfield images from each well
bf_arrays = []

# Read all images, extract the brightfield channel and calculate the mean to correct illumination and remove dust spots
for image in tqdm(images):

    # Read image, apply slicing if needed and return filename and img as a np array
    img, filename = read_image(image, slicing_factor_xy, log=False)

    # Extract brighfield slice
    bf_channel = img[4]

    # Add it to the bf_arrays iterable
    bf_arrays.append(bf_channel)

# Create a stack containing all bf images
bf_stack = np.stack(bf_arrays, axis=0)

# Calculate the median to retain the common structures (spots, illumination)
bf_correction = np.median(bf_stack, axis=0)

viewer.add_image(bf_stack)

viewer.add_image(bf_correction)

In [14]:
del bf_stack

In [ ]:
# Store brightfield correction as .tiff to avoid recalculating it everytime
from tifffile import imwrite, imread

imwrite("./raw_data/test_data_2/bf_correction.tiff",bf_correction)


In [20]:
from tifffile import imread

In [22]:
#TODO: Change to imread(directory_path / "bf_correction.tiff")
bf_correction = imread("./raw_data/test_data_2/bf_correction.tiff")

In [ ]:
# Explore a different image (0 defines the first image in the directory)
image = images[1]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy)

Compressed Array shape: (6, 5032, 5032)


Image analyzed: Plate01_Nuc_Wells-A11__Channel_SD_AF647,SD_RFP,SD_GFP,SD_DAPI,SD_BF,SD_NIR
Original Array shape: (6, 5032, 5032)


In [13]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img)

<Image layer 'img' at 0x1c896643610>

In [16]:
nuclei_labels, flows, styles = model.eval(img[-1:], niter=1000) # need to check the arguments
viewer.add_labels(nuclei_labels)

<Labels layer 'nuclei_labels' at 0x1c896424160>

In [17]:
cytoplasm_labels, flows, styles = model.eval(np.stack((img[[0,1]].sum(axis=0), (img[4] - bf_correction)), axis=0), niter=1000) # need to check the arguments
viewer.add_labels(cytoplasm_labels)

<Labels layer 'cytoplasm_labels' at 0x1c895d13970>

In [18]:
points, details = spotiflow_model.predict(img[0], subpix=True)
viewer.add_image(details.heatmap, colormap="viridis")
viewer.add_points(points, face_color='red')

INFO:spotiflow.model.spotiflow:Will use device: cuda:0
2025-09-01 10:49:26,405 [INFO] Will use device: cuda:0
INFO:spotiflow.model.spotiflow:Predicting with prob_thresh = [0.6], min_distance = 1
2025-09-01 10:49:26,405 [INFO] Predicting with prob_thresh = [0.6], min_distance = 1
INFO:spotiflow.model.spotiflow:Peak detection mode: fast
2025-09-01 10:49:26,405 [INFO] Peak detection mode: fast
INFO:spotiflow.model.spotiflow:Image shape (5032, 5032)
2025-09-01 10:49:26,405 [INFO] Image shape (5032, 5032)
INFO:spotiflow.model.spotiflow:Predicting with (3, 3) tiles
2025-09-01 10:49:26,410 [INFO] Predicting with (3, 3) tiles
INFO:spotiflow.model.spotiflow:Normalizing...
2025-09-01 10:49:26,434 [INFO] Normalizing...
INFO:spotiflow.model.spotiflow:Padding to shape (5040, 5040, 1)
2025-09-01 10:49:26,720 [INFO] Padding to shape (5040, 5040, 1)


Predicting tiles: 100%|██████████| 9/9 [00:03<00:00,  2.86it/s]

INFO:spotiflow.model.spotiflow:Found 1245 spots
2025-09-01 10:49:29,901 [INFO] Found 1245 spots


<Points layer 'points' at 0x1c896424910>

In [ ]:
#TODO: Bacterial detection, infection rate calculation

# Voronoi otsu labeling is very affected by outliers, train APOC object segmenter for best results
import pyclesperanto_prototype as cle   

cle.select_device()  # optional: choose your GPU

# img: 2D/3D grayscale numpy array
bacteria_labels = cle.voronoi_otsu_labeling(
    img[3],             # choose Mycoplasma channel
    spot_sigma=1.0,     # controls object separation (peak detection)
    outline_sigma=1.0   # controls outline smoothness/precision
)

viewer.add_labels(bacteria_labels)

2025-09-01 11:21:38,609 [INFO] build program: kernel 'gaussian_blur_separable_2d' was part of a lengthy uncached source build (assuming cached by ICD) (0.30 s)


<Labels layer 'bacteria_labels' at 0x1c88ec5dd50>